In [138]:
import pandas as pd
import re
import tweepy
import numpy as np

In [200]:
CONSUMER_KEY = ""
CONSUMER_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""

auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET) 
auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


## Read the dataset

In [239]:
df = pd.read_csv('data_tweets01.csv')
df.head(10)

,User ID,Name,User,Text,Created At,Location,Retweet Count,Fav Count
0,1163060730590982144,kabel // kinda ia💘👋,wllyh_,dhlh bingung gmna utk kuliah besok lusa dan st...,2020-09-30 13:26:11,NaN,0,0
1,1289956071537172484,orangcanteeq,siapaakooo,@collegemenfess Kuota bantuan indosat lemot gi...,2020-09-30 13:23:30,NaN,0,0
2,1171388755,nes🌙,lattetrash,"RT @collegemenfess: [cm] Alhamdulillah, Indosa...",2020-09-30 13:21:02,NaN,7,0
3,548904824,Indosat Ooredoo Care,IndosatCare,"@ovienndt Hai Kak Ovie, maaf nih atas kendalan...",2020-09-30 13:06:19,NaN,0,0
4,1201845514259484672,celo,xholtmaan,"bulan kmren gue dpet subsidi kuota dr sekolah,...",2020-09-30 13:05:48,NaN,0,0
5,1201845514259484672,celo,xholtmaan,jadi kan gue lagi mikir nih bantuan kuota dr p...,2020-09-30 13:05:47,NaN,0,0
6,433862698,Firmanto,damar_dyf,Kenapa kuota belajar bantuan Kemendikbud nggak...,2020-09-30 13:05:21,NaN,0,0
7,548904824,Indosat Ooredoo Care,IndosatCare,"@tentaclesTentac Hai Kak, maaf untuk pengalama...",2020-09-30 13:02:39,NaN,0,0
8,1286269100507852801,S T O P I T⁷,bucinJinJay,pada dapat bantuan kuota gtu ga sih ? \r\nTapi...,2020-09-30 13:01:53,NaN,0,0
9,779784931,세정 오빠 🐳,MichellaOey1,"@Telkomsel Kalau kuota bantuan ini, bisa untuk...",2020-09-30 13:00:46,NaN,0,0


In [141]:
df.shape

(4662, 8)

In [142]:
data = pd.DataFrame()
data['id_user'] = df['User ID']
data['user'] = df['User']
data['text'] = df['Text']

In [143]:
data.head()

,id_user,user,text
0,1163060730590982144,wllyh_,dhlh bingung gmna utk kuliah besok lusa dan st...
1,1289956071537172484,siapaakooo,@collegemenfess Kuota bantuan indosat lemot gi...
2,1171388755,lattetrash,"RT @collegemenfess: [cm] Alhamdulillah, Indosa..."
3,548904824,IndosatCare,"@ovienndt Hai Kak Ovie, maaf nih atas kendalan..."
4,1201845514259484672,xholtmaan,"bulan kmren gue dpet subsidi kuota dr sekolah,..."


## Extract mentioned user from the tweet

In [144]:
data['text'][1]

'@collegemenfess Kuota bantuan indosat lemot gitu gasih kalo dipake?? Apa cuma punyaku aja??'

In [145]:
result = re.findall("(^|[^@\w])@(\w{1,15})", data['text'][1])
print(result);

[('', 'collegemenfess')]


In [146]:
def extract(tweet):
    return re.findall("@([a-zA-Z0-9]{1,15})", tweet)

data['mentioned_user'] = data['text'].apply(lambda x: extract(x))

In [147]:
data.head()

,id_user,user,text,mentioned_user
0,1163060730590982144,wllyh_,dhlh bingung gmna utk kuliah besok lusa dan st...,[]
1,1289956071537172484,siapaakooo,@collegemenfess Kuota bantuan indosat lemot gi...,[collegemenfess]
2,1171388755,lattetrash,"RT @collegemenfess: [cm] Alhamdulillah, Indosa...",[collegemenfess]
3,548904824,IndosatCare,"@ovienndt Hai Kak Ovie, maaf nih atas kendalan...",[ovienndt]
4,1201845514259484672,xholtmaan,"bulan kmren gue dpet subsidi kuota dr sekolah,...",[]


### Get ID of mentioned user

In [151]:
def get_id(user_list) :
    ID = []
    try :
        for user in user_list :
            temp = api.get_user(user)
            ID.append(int(temp.id_str))
    except tweepy.TweepError:
        ID.append('kosong')
    return ID

In [152]:
data['id_mentioned_user'] = data['mentioned_user'].apply(lambda x: get_id(x))

In [154]:
data.head()

,id_user,user,text,mentioned_user,id_mentioned_user
0,1163060730590982144,wllyh_,dhlh bingung gmna utk kuliah besok lusa dan st...,[],[]
1,1289956071537172484,siapaakooo,@collegemenfess Kuota bantuan indosat lemot gi...,[collegemenfess],[1046084008742801408]
2,1171388755,lattetrash,"RT @collegemenfess: [cm] Alhamdulillah, Indosa...",[collegemenfess],[1046084008742801408]
3,548904824,IndosatCare,"@ovienndt Hai Kak Ovie, maaf nih atas kendalan...",[ovienndt],[589687384]
4,1201845514259484672,xholtmaan,"bulan kmren gue dpet subsidi kuota dr sekolah,...",[],[]


In [156]:
df_id = data[['id_user', 'id_mentioned_user']]
df_id.head()

,id_user,id_mentioned_user
0,1163060730590982144,[]
1,1289956071537172484,[1046084008742801408]
2,1171388755,[1046084008742801408]
3,548904824,[589687384]
4,1201845514259484672,[]


In [164]:
df_id.shape

(4662, 2)

## Create dataset to be used as edges list

In [173]:
list_id = df_id['id_user']
list_id_mention = df_id['id_mentioned_user']
new_list_id = []
new_list_id_mention = []

for i in range(len(list_id)) :
    if (len(list_id_mention[i]) != 0) :
        for j in range(len(list_id_mention[i])) :
            if (list_id_mention[i][j] != 'kosong') :
                new_list_id.append(int(list_id[i]))
                new_list_id_mention.append(int(list_id_mention[i][j]))

In [241]:
new_id = list(zip(new_list_id,new_list_id_mention))
new_id_df = pd.DataFrame(new_id)
new_id_df.to_csv('preprocessed_id.txt', sep='\t', index=False, header=True)
new_id_df.columns =['source', 'target'] 
new_id_df.to_csv('edge_list.csv') 
new_id_df.head(8)

,source,target
0,1289956071537172484,1046084008742801408
1,1171388755,1046084008742801408
2,548904824,589687384
3,548904824,1246657358844923904
4,779784931,255409050
5,612277839,17600950
6,1306834920992436224,790856528
7,1208708548911173632,917735162300981248


In [175]:
new_id_df.shape

(1562, 2)

## Create dataset to be used as nodes list 

In [212]:
uni = new_id_df.copy()

In [213]:
column_values = uni[[0,1]].values
unique_values =  np.unique(column_values)

In [214]:
len(unique_values)

1445

In [216]:
print(unique_values[0])

5511


In [219]:
unique_name = []
for i in range(len(unique_values)) :
    try : 
        user = api.get_user(unique_values[i]) 
        screen_name = user.screen_name
        unique_name.append([unique_values[i], screen_name])
    except tweepy.TweepError:
        unique_name.append([unique_values[i], 'NOT FOUND'])

In [222]:
len(unique_name)

1445

In [242]:
name_list = pd.DataFrame(unique_name)
name_list.columns =['id', 'label'] 
name_list.to_csv('nodes_list.csv', index=False, header=True)
name_list.tail(8)

,id,label
1437,1309512706127323143,bljaya
1438,1309526327586820097,Meri19509352
1439,1310532840572870656,AhraRaya
1440,1310727904704294917,notracable
1441,1310861203304902658,DwiGarnis6
1442,1310985558831722496,Anakran69858845
1443,1311152902300291076,yulia_aja_
1444,1311231067332829184,mydayxhan
